In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder \
    .appName("fact_product") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/15 07:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/15 07:08:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read \
    .format('iceberg') \
    .load('iceberg.silver.dm_product') 

In [4]:
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- brand_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- favourite_count: integer (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- has_ebook: boolean (nullable = true)
 |-- is_fresh: boolean (nullable = true)
 |-- is_flower: boolean (nullable = true)
 |-- has_buynow: boolean (nullable = true)
 |-- is_gift_card: boolean (nullable = true)
 |-- is_baby_milk: boolean (nullable = true)
 |-- is_acoholic_drink: boolean (nullable = true)
 |-- data_version: string (nullable = true)
 |-- day_ago_created: integer (nullable = true)
 |-- created_date: date (nullable = true)
 |-- ngay_cap_nhat: timestamp (nullable = true)



In [5]:
df = df.withColumn("ngay_cap_nhat", current_timestamp())

df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.gold.fact_product')

In [6]:
spark.stop()